In [1]:
import os
import argparse
import json
import cv2
import keras

from utils.utils import get_yolo_boxes, preprocess_input, decode_netout, correct_yolo_boxes, do_nms
from utils.bbox import draw_boxes

import sys
import numpy as np
import numpy.linalg as la
import tensorflow as tf
from tqdm import tqdm

from keras import optimizers
from keras.utils import np_utils, generic_utils
from keras.models import Sequential,Model,load_model
from keras.layers import Dropout, Flatten, Dense,Input
from keras.applications.resnet50 import ResNet50
from keras.applications.imagenet_utils import preprocess_input
from keras import backend as K
from keras.layers.core import Lambda
from sklearn.preprocessing import normalize
from keras.preprocessing.image import ImageDataGenerator
from keras.initializers import RandomNormal
from sklearn.preprocessing import normalize

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"

/usr/local/lib/python3.6/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
class Feature:
    def __init__(self, feature_vector, class_id, image_data=None):
        self.feature_vector = feature_vector
        self.class_id = class_id
        self.image_data = image_data

In [3]:
keras.backend.set_learning_phase(0)
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
anchors = [10,37, 17,71, 28,104, 28,50, 42,79, 45,148, 70,92, 77,181, 193,310]
labels = ['person']
net_size = 608
obj_thresh, nms_thresh = 0.9, 0.4
snapshot_name = 'snapshots/current_person/yolo3_model.h5'
every_nth = 5

if not os.path.exists(snapshot_name):
    raise FileNotFoundError(snapshot_name)

In [4]:
from yolo import RegressionLayer

infer_model = load_model(snapshot_name, custom_objects={'RegressionLayer': RegressionLayer})

/usr/local/lib/python3.6/dist-packages/keras/engine/saving.py:270: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [5]:
import torch
import torch.nn as nn
import torch.nn.init as init
import torch.nn.functional as F

from resnet import resnet50

def load_state_dict(model, src_state_dict):
    from torch.nn import Parameter

    dest_state_dict = model.state_dict()
    for name, param in src_state_dict.items():
        if name not in dest_state_dict:
            continue
        if isinstance(param, Parameter):
            param = param.data
        try:
            dest_state_dict[name].copy_(param)
        except:
            print("Warning: Error occurs when copying '{}': {}".format(name, str(msg)))

    src_missing = set(dest_state_dict.keys()) - set(src_state_dict.keys())
    if len(src_missing) > 0:
        # print("Keys not found in source state_dict: ")
        pass
    for n in src_missing:
        # print('\t', n)
        pass

    dest_missing = set(src_state_dict.keys()) - set(dest_state_dict.keys())
    if len(dest_missing) > 0:
        print("Keys not found in destination state_dict: ")
        for n in dest_missing:
            print('\t', n)
            pass
        pass


class FeatureModel(nn.Module):
    def __init__(self, local_conv_out_channels=128):
        super(FeatureModel, self).__init__()
        self.base = resnet50(pretrained=True)
        planes = 2048
        self.local_conv = nn.Conv2d(planes, local_conv_out_channels, 1)
        self.local_bn = nn.BatchNorm2d(local_conv_out_channels)
        self.local_relu = nn.ReLU(inplace=True)

    def forward(self, x):
        # shape [N, C, H, W]
        feat = self.base(x)
        global_feat = F.avg_pool2d(feat, feat.size()[2:])
        # shape [N, C]
        global_feat = global_feat.view(global_feat.size(0), -1)
        # shape [N, C, H, 1]
        local_feat = torch.mean(feat, -1, keepdim=True)
        local_feat = self.local_relu(self.local_bn(self.local_conv(local_feat)))
        # shape [N, H, c]
        local_feat = local_feat.squeeze(-1).permute(0, 2, 1)

        return global_feat, local_feat
    
    
def get_feature(model, images):
    ims = torch.autograd.Variable(torch.from_numpy(images).float())
    global_feat, local_feat = model(ims)[:2]
    return global_feat.detach().numpy()


feature_model = FeatureModel()
checkpoint = '/home/svakhreev/PycharmProjects/AlignedReID-Re-Production-Pytorch/snapshots/ckpt.pth'
sd = torch.load(checkpoint, map_location=lambda storage, loc: storage)
load_state_dict(feature_model, sd)

Keys not found in destination state_dict: 
	 state_dicts
	 scores
	 ep


In [7]:
video_reader = cv2.VideoCapture('/mnt/nfs/Videos/peoples_videos/outdoor/bright_by_moscow_time/primorskiy_bulvar.ts.mp4')

all_features = []
obj_thresh, nms_thresh = 0.9, 0.4

def cut_boxes(image, boxes, obj_thresh):
    boxes_imgs = []
    for box in boxes:
        boxes_imgs.append(image[box.ymin:box.ymax, box.xmin:box.xmax, :])
    return boxes_imgs

frames_processed = 0
cv2.namedWindow('image', cv2.WINDOW_KEEPRATIO)
while True:
    read, image = video_reader.read()
    if not read:
        break

    frames_processed += 1

    if frames_processed % every_nth != 0:
        continue

    boxes = get_yolo_boxes(infer_model, image, net_size, net_size, obj_thresh, nms_thresh)
    cv2.imshow('image', draw_boxes(image, boxes, labels, obj_thresh))
    key = cv2.waitKeyEx(1)
    if key == 27:
        break
    
    boxes = [box for box in boxes if box.classes[0] > obj_thresh]
    boxes = [box for box in boxes if box.xmin > 0 and box.ymin > 0]
    cutted_boxes = cut_boxes(image, boxes, obj_thresh)
    data = np.zeros((len(cutted_boxes), 3, 256, 128), dtype=np.float32)
    for i, box in enumerate(cutted_boxes):
        box = cv2.cvtColor(box, cv2.COLOR_BGR2RGB)
        box = cv2.resize(box, (128, 256))
        box = box / 255.
        data[i] = box.transpose(2, 0, 1)
    for i in range((len(data) // 8) + 1):
        if len(data[i*8:i*8+8]) == 0:
            continue
        features = get_feature(feature_model, data[i*8:i*8+8])
        for j, f in enumerate(features):
            all_features.append(Feature(f, 0, image_data=cutted_boxes[i * 8 + j]))
cv2.destroyAllWindows()
len(all_features)

386

In [9]:
def compute_dist(array1, array2, type='euclidean'):
    assert type in ['cosine', 'euclidean']
    if type == 'cosine':
        array1 = normalize(array1, axis=1)
        array2 = normalize(array2, axis=1)
        dist = np.matmul(array1, array2.T)
        return dist
    else:
        # shape [m1, 1]
        square1 = np.sum(np.square(array1), axis=1)[..., np.newaxis]
        # shape [1, m2]
        square2 = np.sum(np.square(array2), axis=1)[np.newaxis, ...]
        squared_dist = - 2 * np.matmul(array1, array2.T) + square1 + square2
        squared_dist[squared_dist < 0] = 0
        dist = np.sqrt(squared_dist)
        return dist

In [10]:
import os
from PIL import Image
import glob

def flat( *nums ):
    return tuple( int(round(n)) for n in nums )

class Size(object):
    def __init__(self, pair):
        self.width = float(pair[0])
        self.height = float(pair[1])

    @property
    def aspect_ratio(self):
        return self.width / self.height

    @property
    def size(self):
        return flat(self.width, self.height)

def cropped_thumbnail(img, size):
    original = Size(img.size)
    target = Size(size)

    if target.aspect_ratio > original.aspect_ratio:
        scale_factor = target.width / original.width
        crop_size = Size( (original.width, target.height / scale_factor) )
        top_cut_line = (original.height - crop_size.height) / 2
        img = img.crop( flat(0, top_cut_line, crop_size.width, top_cut_line + crop_size.height) )
    elif target.aspect_ratio < original.aspect_ratio:
        scale_factor = target.height / original.height
        crop_size = Size( (target.width/scale_factor, original.height) )
        side_cut_line = (original.width - crop_size.width) / 2
        img = img.crop( flat(side_cut_line, 0,  side_cut_line + crop_size.width, crop_size.height) )
        
    return img.resize(target.size, Image.ANTIALIAS)


def get_master_size_and_grid(image_size, images_count):
    w, h = image_size
    for i in range(100):
        if (i * i) > images_count:
            return (w * i, h * i), i

        
def create_sprite(path, images, size=(150, 150)):
    images = [Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB), 'RGB') for image in images]
    images = [cropped_thumbnail(image, size) for image in images]

    master_size, grid = get_master_size_and_grid(size, len(images))

    master = Image.new(
        mode='RGBA',
        size=(master_size[0], master_size[1]),
        color=(0, 0, 0, 0))

    image_index = 0
    for i in range(grid):
        for j in range(grid):
            if image_index >= len(images):
                current_image = Image.new('RGBA', size, color=(0, 0, 0, 0))
            else:
                current_image = images[image_index] 
            master.paste(current_image, (j * size[1], i * size[0]))
            image_index += 1
        
    master.save(os.path.join(path, 'sprite.png'))
    return os.path.join(path, 'sprite.png')

In [11]:
import cv2
import shutil
from tensorflow.contrib.tensorboard.plugins import projector

log_dir = './logs/'
test_images_dir = os.path.join(log_dir, 'test_images')

session = K.get_session()

if not os.path.exists(log_dir):
    os.mkdir(log_dir)

if not os.path.exists(test_images_dir):
    os.mkdir(test_images_dir)
else:
    shutil.rmtree(log_dir)
    os.mkdir(log_dir)
    os.mkdir(test_images_dir)

embedding_var = tf.Variable(np.array([f.feature_vector for f in all_features]),
                            name='images_embedding', dtype=tf.float32)
session.run(tf.global_variables_initializer())

config = projector.ProjectorConfig()
embedding = config.embeddings.add()
embedding.tensor_name = embedding_var.name

sprite_element_size = (50, 50)
create_sprite(test_images_dir, [f.image_data for f in all_features], size=sprite_element_size)
embedding.sprite.image_path = '/home/svakhreev/PycharmProjects/keras-yolo3/logs/test_images/sprite.png'
embedding.sprite.single_image_dim.extend(sprite_element_size)

summary_writer = tf.summary.FileWriter(log_dir)
projector.visualize_embeddings(summary_writer, config)

saver = tf.train.Saver()
saver.save(session, os.path.join(log_dir, "model.ckpt"))

'./logs/model.ckpt'